In [2]:
import tensorflow as tf

/mnt/md0/home/penglish/anaconda3/lib/python3.6/site-packages/numexpr/cpuinfo.py:42: UserWarning: [Errno 2] No such file or directory: 'uname'
  warnings.warn(str(e), UserWarning, stacklevel=stacklevel)


In [1]:
# Parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100
display_step = 1

In [3]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [4]:
n_input = 784 # MNIST data input (img shape: 28*28)
n_classes = 10 # MNIST total classes (0-9 digits)

In [5]:
# tf Graph input
x = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float", [None, n_classes])

# Perceptron

For $\mathbf{X}$ the MNIST digits and $y$ labels

$$
\hat{y}_j = \sum_{x \in \mathbf{X}} w_j x + b_j
$$

In [11]:
hidden_weights = 256

w = tf.Variable(tf.random_normal([n_input, hidden_weights]))
b = tf.Variable(tf.random_normal([hidden_weights]))

w2 = tf.Variable(tf.random_normal([hidden_weights, hidden_weights]))
w3 = tf.Variable(tf.random_normal([hidden_weights, n_classes]))

input_layer = tf.add(tf.matmul(x, w), b)
hidden = tf.matmul(input_layer, w2)
perceptron = tf.matmul(hidden, w3)

In [12]:
# Loss & Optimizer

cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=perceptron, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy)

In [13]:
# Initialize the variables

init = tf.global_variables_initializer()

In [14]:
with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(mnist.train.num_examples/batch_size)
        
        for i in range(total_batch):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            _, c = sess.run([optimizer, cross_entropy], feed_dict={x: batch_x, y: batch_y})
        
            avg_cost += c / total_batch
            
        if epoch % display_step == 0:
            print("Epoch:", "%04d" % (epoch+1), "cost=", "{:.9f}".format(avg_cost))
            
    print("Finished")
    
    
    correct_prediction = tf.equal(tf.argmax(perceptron, 1), tf.argmax(y, 1))
    
    # Accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print("Accuracy:", accuracy.eval({x: mnist.test.images, y: mnist.test.labels}))

Epoch: 0001 cost= 281.901188726
Epoch: 0002 cost= 93.115760567
Epoch: 0003 cost= 69.666210016
Epoch: 0004 cost= 55.836551976
Epoch: 0005 cost= 47.883651159
Epoch: 0006 cost= 41.284338692
Epoch: 0007 cost= 36.700200598
Epoch: 0008 cost= 33.038762716
Epoch: 0009 cost= 31.282793785
Epoch: 0010 cost= 28.052685062
Epoch: 0011 cost= 26.393186235
Epoch: 0012 cost= 24.659118223
Epoch: 0013 cost= 23.346494285
Epoch: 0014 cost= 21.734469367
Epoch: 0015 cost= 20.599538615
Finished
Accuracy: 0.8888


# Perceptron (No cheating, mostly)

Perceptron loss

$$
\text{loss} := \min(0, -y * (\mathbf{X}w + b))
$$

- Using our own perceptron loss
- Using the gradient descent optimizer (partial cheating, but useful in practice)

In [15]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
import numpy as np

classification_x, classification_y = make_classification(1000, n_features=100, n_informative=30)
classification_y = np.expand_dims(classification_y, axis=1)

print(classification_x.shape, classification_y.shape)

train_x, test_x, train_y, test_y = train_test_split(classification_x, classification_y, test_size=0.33)

(1000, 100) (1000, 1)


In [16]:
n_input = classification_x.shape[1]
n_classes = classification_y.shape[1]

In [17]:
x = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float", [None, n_classes])

In [18]:
w = tf.Variable(tf.random_normal([n_input, n_classes]))
b = tf.Variable(tf.random_normal([n_classes]))

perceptron = tf.add(tf.matmul(x, w), b)

In [19]:
perceptron_loss = tf.reduce_mean(tf.maximum(0., -y * tf.add(tf.matmul(x, w), b)))

In [20]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(perceptron_loss)

In [22]:
init = tf.global_variables_initializer()

In [23]:
training_epochs = 100
learning_rate = 0.001
display_step = 10

In [24]:
with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(train_x.shape[0]/batch_size)
        
        for i in range(total_batch):
            start = i * batch_size
            end = start + batch_size
            batch_x = train_x[start:end]
            batch_y = train_y[start:end]

            _, c = sess.run([optimizer, perceptron_loss], feed_dict={x: batch_x, y: batch_y})
        
            avg_cost += c / total_batch
            
        if epoch % display_step == 0:
            print("Epoch:", "%04d" % (epoch+1), "cost=", "{:.9f}".format(avg_cost))
            
    print("Finished")
    
    
    correct_prediction = tf.equal(tf.argmax(perceptron, 1), tf.argmax(y, 1))
    
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print("Accuracy:", accuracy.eval({x: test_x, y: test_y}))

Epoch: 0001 cost= 6.091614803
Epoch: 0011 cost= 5.740135670
Epoch: 0021 cost= 5.396558046
Epoch: 0031 cost= 5.062470039
Epoch: 0041 cost= 4.754018625
Epoch: 0051 cost= 4.459690650
Epoch: 0061 cost= 4.183171590
Epoch: 0071 cost= 3.931954304
Epoch: 0081 cost= 3.692603827
Epoch: 0091 cost= 3.465417663
Finished
Accuracy: 1.0


# Perceptron (No cheating, for real)

- Attempting to write our own update/optimize at each step

In [25]:
margin = 0.1

In [26]:
x = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float", [None, n_classes])

w = tf.Variable(tf.random_normal([n_input, n_classes]))

In [28]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(train_x.shape[0]/batch_size)
        
        for i in range(total_batch):
            start = i * batch_size
            end = start + batch_size
            batch_x = train_x[start:end]
            batch_y = train_y[start:end]
            
            loss = tf.reduce_mean(tf.maximum(0., -y * tf.matmul(x, w), b))
            is_mistake = tf.less_equal(y * tf.matmul(x, w), margin)
            eta = (margin - (y[is_mistake] * tf.matmul(x[is_mistake], w))) / (tf.matmul(x[is_mistake], tf.transpose(x[is_mistake])) + 1)
            update = tf.assign(w, eta * tf.matmul(x[is_mistake], y[is_mistake]))
    
            _, c = sess.run([update, loss])
            
            avg_cost += c / total_batch
            
        if epoch % display_step == 0:
            print("Epoch:", "%04d" % (epoch+1), "cost=", "{:.9f}".format(avg_cost))
            
    print("Finished")
    
    
    correct_prediction = tf.equal(tf.argmax(perceptron, 1), tf.argmax(y, 1))
    
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print("Accuracy:", accuracy.eval({x: test_x, y: test_y}))

TypeError: expected string or bytes-like object